# Stock Chart Pattern Recognition with Deep Learning
## CRISP-DM Methodology

1. Business Understanding

2. Data Understanding (ความเข้าใจในข้อมูล)
การเก็บข้อมูล: รวบรวมข้อมูลราคาหุ้นย้อนหลัง (OHLC: Open, High, Low, Close) และ Volume การสำรวจข้อมูล: ดูลักษณะกราฟแท่งเทียน (Candlestick) หรือกราฟเส้น เพื่อระบุช่วงเวลาที่เคยเกิด HS ขึ้นจริงๆ ในอดีต เพื่อใช้เป็นแนวทางในการทำ Labeling

🟦 Cell 1: Install & Import Library

In [ ]:
# ถ้าใช้ครั้งแรก ให้ uncomment
# !pip install settrade-v2 cassandra-driver mplfinance tensorflow scikit-learn scipy python-dotenv

import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv

from cassandra.cluster import Cluster
from settrade_v2 import Investor

from scipy.signal import find_peaks
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras import layers


In [ ]:
🟦 Cell 2: Load ENV & Connect SETTRADE

In [ ]:
load_dotenv()

investor = Investor(
    app_id=os.getenv("SETTRADE_APP_ID"),
    app_secret=os.getenv("SETTRADE_APP_SECRET"),
    broker_id=os.getenv("SETTRADE_BROKER_ID"),
    app_code=os.getenv("SETTRADE_APP_CODE"),
    is_auto_queue=False
)

market = investor.MarketData()
print("✅ Connected to SETTRADE API")


🟦 Cell 3: Connect Cassandra

In [ ]:
cluster = Cluster(['127.0.0.1'])
session = cluster.connect('stock_data')

print("✅ Connected to Cassandra")

🟦 Cell 4: Fetch OHLCV from SETTRADE

In [ ]:
df = market.get_candlestick(
    symbol="PTT",
    timeframe="1D",
    limit=600
)

df.head()


insert_stmt = session.prepare("""
INSERT INTO ohlcv_by_symbol
(symbol, timeframe, candle_time, open, high, low, close, volume)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
""")

for _, r in df.iterrows():
    session.execute(insert_stmt, (
        "PTT", "1D",
        r['datetime'],
        float(r['open']),
        float(r['high']),
        float(r['low']),
        float(r['close']),
        float(r['volume'])
    ))


🟦 Cell 6: Load Data from Cassandra

In [ ]:
rows = session.execute("""
SELECT * FROM ohlcv_by_symbol
WHERE symbol='PTT' AND timeframe='1D'
""")

df = pd.DataFrame(rows).sort_values("candle_time")
df.head()


In [ ]:
3. Data Preparation (การเตรียมข้อมูล)
Data Labeling: ติดป้ายกำกับ (Label) ว่าช่วงใดของกราฟที่เป็น "Head", "Left Shoulder", "Right Shoulder" และ "Neckline" Image 

Transformation: หากใช้โมเดลประเภท CNN (Convolutional Neural Network) ต้องแปลงข้อมูลตัวเลขราคาให้เป็น รูปภาพกราฟ (เช่น 2D Candlestick Chart) เพื่อให้โมเดลประมวลผลเหมือนสายตามนุษย์ 

Normalization: ปรับสัดส่วนราคา (Scaling) ให้เหมาะสมเพื่อให้โมเดลเรียนรู้ได้ดีขึ้น

🟦 Cell 7: Sliding Window

In [ ]:
WINDOW = 60

def create_windows(df):
    windows = []
    for i in range(len(df) - WINDOW):
        windows.append(df.iloc[i:i+WINDOW])
    return windows

windows = create_windows(df)
len(windows)


🟦 Cell 8: Normalization

In [ ]:
def normalize_window(w):
    base = w['close'].iloc[0]
    w[['open','high','low','close']] /= base
    return w


🟦 Cell 9: Labeling (Rule-based)

In [ ]:
def label_hs(close):
    peaks, _ = find_peaks(close, distance=5)
    if len(peaks) < 3:
        return 0

    l, h, r = peaks[-3:]
    return int(close[h] > close[l] and close[h] > close[r])


In [ ]:
4. Modeling (การสร้างโมเดล) เลือกโมเดล: เลือกใช้โครงสร้าง Deep Learning ที่เหมาะสม เช่น: CNN: เพื่อตรวจจับรูปทรงจากภาพกราฟ 

LSTM / GRU: หากต้องการวิเคราะห์ข้อมูลแบบอนุกรมเวลา (Time Series) เพื่อดูการเรียงตัวของราคา 

Training: ฝึกสอนโมเดลด้วยชุดข้อมูลที่ติด Label ไว้แล้ว โดยอาจใช้เทคนิค Data Augmentation เพื่อเพิ่มจำนวนข้อมูลตัวอย่าง และ จะใช้ Accuracy, Precision, Recall และ Confusion Matrix บนชุดข้อมูล Validation Set เพื่อเปรียบเทียบว่า Algorithm ไหนให้ผลลัพธ์ทางเทคนิคดีที่สุด

🟦 Cell 10: Build Dataset

In [ ]:
X, y = [], []

for w in windows:
    w = normalize_window(w.copy())
    X.append(w[['open','high','low','close','volume']].values)
    y.append(label_hs(w['close'].values))

X = np.array(X)
y = np.array(y)

X.shape, y.shape


🟦 Cell 11: Train / Validation Split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, shuffle=False
)


🟦 Cell 12: LSTM Model

In [ ]:
model = tf.keras.Sequential([
    layers.LSTM(64, return_sequences=True, input_shape=(60,5)),
    layers.LSTM(32),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


🟦 Cell 13: Training

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32
)


5. Evaluation (การประเมินผล)
Technical Metrics: ตรวจสอบค่า Loss, Precision, และ Recall ว่าโมเดลตรวจจับ HS พลาดบ่อยแค่ไหน (False Positive) หรือมองข้ามรูปแบบที่เกิดขึ้นจริงไปหรือไม่ Backtesting: นำโมเดลไปทดสอบกับข้อมูลหุ้นจริงในอดีตที่โมเดลไม่เคยเห็น (Unseen Data) 

🟦 Cell 14: Evaluation

In [ ]:
y_pred = (model.predict(X_val) > 0.5).astype(int)

print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))


🟦 Cell 15: Backtesting

In [ ]:
capital = 100000
position = 0

prices = df['close'].values[-len(y_pred):]

for i, signal in enumerate(y_pred):
    if signal == 1:
        position = -1
    capital += position * (prices[i+1] - prices[i])

capital


6. Deployment (การนำไปใช้งานจริง)
System Integration: นำโมเดลไปเชื่อมต่อกับโปรแกรมเทรดหรือ Dashboard เพื่อส่งสัญญาณแจ้งเตือน (Alert) แบบ Real-time เมื่อพบรูปแบบ HS กำลังก่อตัว Monitoring: ติดตามผลการทำงานอย่างต่อเนื่อง เพราะพฤติกรรมตลาดหุ้นอาจเปลี่ยนแปลงไปตามสภาวะเศรษฐกิจ ซึ่งอาจต้องมีการ Retrain โมเดลใหม่ในอนาคต

🟦 Cell 16: Real-time Detection

In [ ]:
def realtime_detect(symbol="PTT"):
    df = market.get_candlestick(symbol, "1D", 60)
    w = normalize_window(df.copy())
    X = w[['open','high','low','close','volume']].values
    prob = model.predict(X.reshape(1,60,5))[0][0]
    return prob

realtime_detect("PTT")


🟦 Cell 17: Save Signal to Cassandra

In [ ]:
session.execute("""
INSERT INTO hs_signal
(symbol, timeframe, detect_time, probability)
VALUES (%s, %s, toTimestamp(now()), %s)
""", ("PTT", "1D", float(realtime_detect())))
